### Multi Layer Perceptron in KERAS

Simple MLP for MNIST image data prediction to showcase Keras basic functionality

In [64]:
import os
import struct
import numpy as np
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2
 
    return images, labels

In [65]:
X_train, y_train = load_mnist('image_datasets', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

Rows: 60000, columns: 784


In [66]:
X_test, y_test = load_mnist('image_datasets', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 10000, columns: 784


Data transformations

In [67]:
#Mean centering:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

In [68]:
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [69]:
del X_train, X_test

In [70]:
print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [71]:
import tensorflow as tf
import tensorflow.contrib.keras as keras

In [72]:
np.random.seed(1)
tf.set_random_seed(1)

In [73]:
#One hot encoding target feature i.e. digit class 1-9:
y_train_onehot = keras.utils.to_categorical(y_train)
print('First 3 labels: ', y_train[:3])
print('\nFirst 3 labels: (one-hot):\n', y_train_onehot[:3])

First 3 labels:  [5 0 4]

First 3 labels: (one-hot):
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


Model Creation

In [74]:
#To create a model one can easily initialize the Sequential object:
model = keras.models.Sequential()

In [75]:
#Adding INPUT layer - NB input_dim is equal to the shape of the input dataset
model.add(
    keras.layers.Dense(
        units=50,
        input_dim=X_train_centered.shape[1], 
        kernel_initializer='glorot_uniform', 
        bias_initializer='zeros', 
        activation='tanh'))

In [76]:
#Adding a hidden layer - NB the input_dim is equal to the number of units from the previous layer
model.add(
    keras.layers.Dense(
        units=50,
        input_dim=50, #changed to 50
        kernel_initializer='glorot_uniform', 
        bias_initializer='zeros', 
        activation='tanh'))

In [77]:
#Adding the output layer - NB the number of units is equal to the number of class categories 
#in the one hot encoded y vector
model.add(
    keras.layers.Dense(
        units=y_train_onehot.shape[1], #same number 
        input_dim=50, #Same number as units in previous layer
        kernel_initializer='glorot_uniform', 
        bias_initializer='zeros', 
        activation='softmax'))

In [78]:
#Finally, setting the optimizer with SGD
sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=0.9)

In [79]:
#Last step: compiling the model:
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy')

The categorical cross entropy is the cost function for logistic regression generalized for multiclass problems

In [80]:
#Fitting the model:
history = model.fit(X_train_centered, y_train_onehot, batch_size=100, epochs=50, verbose=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 2s 34us/step - loss: 0.8981 - val_loss: 0.4544
Epoch 2/50
54000/54000 [==============================] - 1s 28us/step - loss: 0.4487 - val_loss: 0.3316
Epoch 3/50
54000/54000 [==============================] - 2s 31us/step - loss: 0.3627 - val_loss: 0.2832
Epoch 4/50
54000/54000 [==============================] - 1s 27us/step - loss: 0.3190 - val_loss: 0.2544
Epoch 5/50
54000/54000 [==============================] - 1s 27us/step - loss: 0.2904 - val_loss: 0.2348
Epoch 6/50
54000/54000 [==============================] - 2s 37us/step - loss: 0.2688 - val_loss: 0.2207
Epoch 7/50
54000/54000 [==============================] - 2s 29us/step - loss: 0.2516 - val_loss: 0.2089
Epoch 8/50
54000/54000 [==============================] - 2s 30us/step - loss: 0.2373 - val_loss: 0.1993
Epoch 9/50
54000/54000 [==============================] - 2s 30us/step - loss: 0.2249 - val_loss: 0.1908
Epoch 

The model is converging to a training loss of 0.07 and a validation loss of 0.11. Let's see if we can do better with different hyperparameters

In [81]:
#Fitting the model:
history = model.fit(X_train_centered, y_train_onehot, batch_size=50, epochs=60, verbose=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/60
54000/54000 [==============================] - 3s 49us/step - loss: 0.0722 - val_loss: 0.1166
Epoch 2/60
54000/54000 [==============================] - 3s 51us/step - loss: 0.0696 - val_loss: 0.1160
Epoch 3/60
54000/54000 [==============================] - 3s 48us/step - loss: 0.0671 - val_loss: 0.1165
Epoch 4/60
54000/54000 [==============================] - 2s 44us/step - loss: 0.0647 - val_loss: 0.1152
Epoch 5/60
54000/54000 [==============================] - 2s 44us/step - loss: 0.0622 - val_loss: 0.1166
Epoch 6/60
54000/54000 [==============================] - 2s 45us/step - loss: 0.0599 - val_loss: 0.1161
Epoch 7/60
54000/54000 [==============================] - 2s 45us/step - loss: 0.0579 - val_loss: 0.1153
Epoch 8/60
54000/54000 [==============================] - 3s 49us/step - loss: 0.0559 - val_loss: 0.1147
Epoch 9/60
54000/54000 [==============================] - 3s 48us/step - loss: 0.0540 - val_loss: 0.1139
Epoch 

The model above reached minimum validation loss around the 39th epoch. At that stage, it reached a training loss of 0.02. We are overfitting a little bit. Most importantly, it seems that the validation loss is unchanged from the previous model. 

Further hyperparameter tuning would be needed to finalize the model. We could change values for momentum, weight decay, number of hidden units, and learning rage. Now we create some model evaluation metrics.

In [82]:
y_train_pred = model.predict_classes(X_train_centered, verbose=0)

In [83]:
correct_preds = np.sum(y_train == y_train_pred, axis=0)

In [84]:
train_acc = correct_preds / y_train.shape[0]

In [85]:
print('First 3 predictions: ', y_train_pred[:3])

First 3 predictions:  [5 0 4]


In [86]:
print('Training accuracy: {}%'.format(train_acc*100))

Training accuracy: 99.605%


In [87]:
y_test_pred = model.predict_classes(X_test_centered, verbose=0)

In [88]:
correct_preds = np.sum(y_test == y_test_pred, axis=0)

In [89]:
test_acc = correct_preds / y_test.shape[0]

In [90]:
print('Test Accuracy: %.2f%%' %(test_acc*100))

Test Accuracy: 96.21%
